## Transformação camada prata: sales_salesorderdetail

In [0]:
%run ../Config/DeltaFunctions

In [0]:
%run ../Config/LogProcessamento

In [0]:
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, StringType, TimestampType, StructType, StructField, BooleanType
)

In [0]:
# Habilitar a evolução automática de esquemas
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")
spark.sql('USE CATALOG hive_metastore')

# Informações da Tabela Fonte
source_table = "sales_salesorderdetail"
source_database = "adventure_works_bronze"
bronze_source_table = spark.read.table(f"{source_database}.{source_table}")

# Informações da Tabela Destino (target)
target_table_name = "sales_salesorderdetail"
target_database = "adventure_works_silver"
target_table = f"{target_database}.{target_table_name}"

primary_keys = ["CurrencyCode"]

In [0]:
expected_schema = StructType([
    StructField("CustomerID", IntegerType(), False),             # int IDENTITY(1,1) NOT NULL
    StructField("PersonID", IntegerType(), True),                # int NULL
    StructField("StoreID", IntegerType(), True),                 # int NULL
    StructField("TerritoryID", IntegerType(), True),             # int NULL
    StructField("AccountNumber", StringType(), True),            # Calculado: 'AW' + LeadingZeros(CustomerID)
    StructField("rowguid", StringType(), False),                 # uniqueidentifier NOT NULL
    StructField("ModifiedDate", TimestampType(), False)          # datetime NOT NULL
])

In [0]:
def transform_Customer(Customer: DataFrame) -> DataFrame:
    '''
    Transformação da tabela: Customer

    Parâmetros:
        Customer (DataFrame): DataFrame contendo os dados da tabela Customer

    Retorna:
        DataFrame: O DataFrame resultante após a transformação e deduplicação.
    '''

    # Calcula o AccountNumber baseado no CustomerID
    Customer = Customer.withColumn(
        'AccountNumber',
        F.when(F.col('AccountNumber').isNull(), F.concat(F.lit('AW'), F.format_string('%09d', F.col('CustomerID')))).otherwise(F.col('AccountNumber'))
    )

    # Define valores padrão para campos que podem ser nulos
    Customer = Customer.withColumn(
        'rowguid',
        F.when(F.col('rowguid').isNull(), F.expr('uuid()')).otherwise(F.col('rowguid'))
    )

    Customer = Customer.withColumn(
        'ModifiedDate',
        F.when(F.col('ModifiedDate').isNull(), F.current_timestamp()).otherwise(F.col('ModifiedDate'))
    )

    # Define a função de janela para deduplicar com base nas chaves primárias
    window_spec = Window.partitionBy('CustomerID').orderBy(F.col('ModifiedDate').desc())
    Customer = Customer.withColumn('row_num', F.row_number().over(window_spec))

    # Filtra para manter apenas a primeira linha em cada partição (sem duplicatas)
    Customer = Customer.filter(F.col('row_num') == 1).drop('row_num')

    # Seleção final com CAST explícito dos tipos de dados
    Customer = Customer.select(
        F.col('CustomerID').cast(IntegerType()).alias('CustomerID'),
        F.col('PersonID').cast(IntegerType()).alias('PersonID'),
        F.col('StoreID').cast(IntegerType()).alias('StoreID'),
        F.col('TerritoryID').cast(IntegerType()).alias('TerritoryID'),
        F.col('AccountNumber').cast(StringType()).alias('AccountNumber'),
        F.col('rowguid').cast(StringType()).alias('rowguid'),
        F.col('ModifiedDate').cast(TimestampType()).alias('ModifiedDate')
    )

    return Customer

## Aplicar Transformação

In [0]:
# Estrutura do log para registrar informações sobre o processo
log_data = {
    "log_tabela": source_table,
    "log_camada": "Silver",
    "log_origem": "adventure_works_bronze",
    "log_destino": "adventure_works_silver",
}

# Registra o início do processo
addlog(**log_data, log_status='Início', atualizacao=0)

try:
    # Realiza a transformação dos dados
    transformed_df = transform_Customer(Customer = bronze_source_table)
    
    # Verifica rapidamente o número de linhas e o schema do DataFrame
    row_count = transformed_df.count()
    transformed_df.printSchema()

    # Validação do schema
    is_schema_valid = _validate_schema(transformed_df, expected_schema)
    if is_schema_valid:
        addlog(**log_data, log_status='Sucesso', atualizacao=1)
        print("O schema do DataFrame está correto.")
    else:
        raise ValueError("Schema validation failed.")
    
except Exception as e:
    # Registra erro caso ocorra uma exceção
    addlog(**log_data, log_status='Falha', atualizacao=1)
    print(f"Erro ao processar a tabela: {str(e)}")
    raise  

# Se o schema for válido, realiza o upsert
_upsert_silver_table(transformed_df, target_table, primary_keys, not_matched_by_source_action="DELETE")
